Collaborative Filtering using Consine Similarity, Weighted Sum of Others' Ratings

In [6]:
import pandas as pd
import numpy as np
import math

In [7]:
#make Arbitrary User-Item Matrix
rating={
    "A":{"블랙머니":5,"카운트다운":4,"나이브스아웃":1},
    "B":{"겨울왕국":2,"카운트다운":1,"나이브스아웃":2},
    "C":{"겨울왕국":1,"쥬만지":4,"나이브스아웃":4,"카운트다운":4},
    "D":{"겨울왕국":3,"쥬만지":2,"블랙머니":3,"나이브스아웃":1},
    "E":{"쥬만지":4,"블랙머니":5,"카운트다운":2}
}
DF=pd.DataFrame(rating).T
DF

,겨울왕국,나이브스아웃,블랙머니,쥬만지,카운트다운
A,NaN,1.0,5.0,NaN,4.0
B,2.0,2.0,NaN,NaN,1.0
C,1.0,4.0,NaN,4.0,4.0
D,3.0,1.0,3.0,2.0,NaN
E,NaN,NaN,5.0,4.0,2.0


In [8]:
#get Cosine Similarity
def Norm(i):
    norm=math.sqrt(sum(np.square(i)))
    return norm

def Dot(i,j):
    product=np.dot(i,j)
    return product

def Cosine(rating,A,B):
    #get Co_rating_vector
    A_rating=[]
    B_rating=[]
    for i in rating.get(A).keys():
        if rating.get(B).get(i) != None:
            A_rating.append(rating.get(A).get(i))
            B_rating.append(rating.get(B).get(i))
    #get Similarity
    sim=Dot(A_rating,B_rating)/(Norm(A_rating)*Norm(B_rating))
    return sim


In [9]:
#Weighted Sum of Others' Ratings
#Preference of User A on Item I
def Predict(rating,A,I):
    user=list(rating.keys())
    user.remove(A)
    #Estimate Rating of User A
    Avg_rating_A=np.average(list(rating.get(A).values()))
    Numerator=[]
    Denominator=[]
    for B in user:
        rating_B_I=rating.get(B).get(I)
        if rating_B_I !=None:
            #Estimaet Rating of Others
            Avg_rating_B=np.average(list(rating.get(B).values()))
            #Similarity between User A and Other
            sim=Cosine(rating,A,B)
            Numerator.append((rating_B_I-Avg_rating_B)*sim)
            Denominator.append(math.fabs(sim))
    pre_rating_A_I=Avg_rating_A+sum(Numerator)/sum(Denominator)
    return pre_rating_A_I

In [10]:
#Make new User-Itme Matrix using Collaborative Filtering
items=DF.columns
users=DF.index
new_rating={}
for i in DF.index:
    user_rating={}
    for j in DF.columns:
        user_rating[j]=Predict(rating,i,j)
    new_rating[i]=user_rating

newDF=pd.DataFrame(new_rating).T
newDF

,겨울왕국,나이브스아웃,블랙머니,쥬만지,카운트다운
A,2.946132,3.181249,4.369725,3.587737,2.771220
B,0.988789,0.806770,2.880239,1.932294,1.413522
C,3.775872,2.150900,4.534909,3.336097,2.657665
D,1.451120,1.672041,3.749668,2.754974,1.950691
E,3.308755,3.043001,4.865798,3.904441,3.901674
